# Projet  **<span style="color: #DC143C">Quiz generator </span>** 💡 - Python pour le data-scientist

#### Auteurs : Adrien Servière, Mélissa Tamine.

L'objectif de ce notebook est de présenter le projet que nous avons effectué dans le cadre de l'unité d'enseignement "Python pour le data-scientist" dispensée à l'ENSAE. Ce projet a été elaboré de manière libre et comporte, comme attendu, un **jeu de données**, une partie **visualisation** et une partie **modélisation**. 

In [5]:
from src.data.data_format import *

In [34]:
contexte = Context(text="L'ENSAE Paris est spécialisée dans les mathématiques appliquées, la statistique, la science des données, et l'économie.",
                   meta=dict(url="https://fr.wikipedia.org/wiki/%C3%89cole_nationale_de_la_statistique_et_de_\
                   l%27administration_%C3%A9conomique",
                   document_title='ENSAE'))
reponse = Answer(text="École polytechnique", context=contexte)
question = Question(text="De quelle grande école l'ENSAE est-elle une école d'application ?", gold_answers=[reponse], predicted_answers=[reponse])

In [35]:
print(f'Contexte :\n\n{reponse.context.text}\n')
print(f'Question : {question.text}\n')
print('Réponses : \n' + '\n'.join(f'\t{i + 1}: ' + str(ga.text) for i, ga in enumerate(question.gold_answers)))

Contexte :

L'ENSAE Paris est spécialisée dans les mathématiques appliquées, la statistique, la science des données, et l'économie.

Question : De quelle grande école l'ENSAE est-elle une école d'application ?

Réponses : 
	1: École polytechnique
